In [ ]:
import numpy as np
from ase import Atoms, Atom
from ase.io import write
from ase.data import chemical_symbols
from sklearn import manifold, datasets
from sklearn.decomposition import PCA
from openbabel import pybel as pb
from openbabel import openbabel as ob
from ase.visualize import view

from rdkit.Chem.rdmolfiles import MolFromSmiles,MolToMolFile
from rdkit import Chem
from rdkit.Chem import AllChem

## Some functions to create a 3D structure from a SMILE string

In [ ]:
def __ase2xyz__(atoms):
    """
    Prepare a XYZ string from an ASE atoms object.
    """
    # Implementation detail: If PBC should be implemented, the
    # write to xyz needs to be changed to include cell etc.
    atoms.pbc=False
    if any(atoms.get_pbc()):
        raise RuntimeError("Detected PBCs. Not supported (yet)!")
    num_atoms = len(atoms)
    types = atoms.get_chemical_symbols()
    all_atoms = zip(types, atoms.get_positions())
    a_str = str(num_atoms) + "\n" + "\n"
    for atom in all_atoms:
        a_str += atom[0] + " " + " ".join([str(x) for x in atom[1]]) + "\n"
    return a_str


def convert_ase2pybel(atoms):
    """
    Convert an ASE atoms object to pybel (openBabel) molecule.
    The ordering of the Atoms is identical.

    Parameters
    ----------
    atoms : ase.Atoms
        The ASE atoms object

    Returns
    -------
    pymol :
        The pybel molecule.
    """
    atoms.pbc=False
    a_str = __ase2xyz__(atoms)
    pymol = pb.readstring("xyz", a_str)

    return pymol

def convert_ase2rdkit(atoms, removeHs=False):
    """
    Convert an ASE atoms object to rdkit molecule.
    The ordering of the Atoms is identical.


    Important: Implemented only for clusters, not PBC!
    rdkit does not keep xyz coordinates, therefore
    a backconversion is not possible yet.

    Parameters
    ----------
    atoms : ase.Atoms
        The ASE atoms object
    removeHs : Bool
        If True, remove all H atoms from molecule.

    Returns
    -------
    mol : rdkit.Chem.rdchem.Mol
        The rdkit molecule object.
    """
    a_str = __ase2xyz__(atoms)
    pymol = pb.readstring("xyz", a_str)
    mol = pymol.write("mol")
    mol = Chem.MolFromMolBlock(mol, removeHs=removeHs)
    return mol

def pybel2ase(mol):  
    asemol = Atoms()
    species=[chemical_symbols[atm.atomicnum] for atm in mol.atoms]
    pos=np.asarray([atm.coords for atm in mol.atoms])
    pca = PCA(n_components=3)
    posnew=pca.fit_transform(pos)
    atoms = Atoms(species, positions=posnew)
    sys_size = np.ptp(atoms.positions,axis=0)
    atoms.pbc=True
    atoms.cell = sys_size + 10
    atoms.center()
    
    return atoms

def rdkit2ase(m):
    pos = m.GetConformer().GetPositions()
    natoms = m.GetNumAtoms()
    species = [m.GetAtomWithIdx(j).GetSymbol() for j in range(natoms)]                
#Get the principal axes and realign the molecule
    pca = PCA(n_components=3)
    pca.fit(pos)
    posnew=pca.transform(pos)        
#Set the z to 0.0       
    #posnew[:,2]=0.0
    atoms = Atoms(species, positions=posnew)  
    sys_size = np.ptp(atoms.positions,axis=0)
    atoms.pbc=True
    atoms.cell = sys_size + 10
    atoms.center()
        
    return atoms


def pybel_opt(smile,steps):
    obconversion = ob.OBConversion()
    obconversion.SetInFormat('smi')
    obmol = ob.OBMol()
    obconversion.ReadString(obmol,smile)
    
    pbmol = pb.Molecule(obmol)
    pbmol.make3D(forcefield="uff", steps=50)
    
    pbmol.localopt(forcefield="gaff", steps=200)
    pbmol.localopt(forcefield="mmff94", steps=100)
    
    f_f = pb._forcefields["uff"]
    f_f.Setup(pbmol.OBMol)
    f_f.ConjugateGradients(steps, 1.0e-9)
    f_f.GetCoordinates(pbmol.OBMol)
    #print(f_f.Energy(), f_f.GetUnit())
    
    return pybel2ase(pbmol)

def try_rdkit(smile):
    test = Chem.MolFromSmiles(smile)
    test = Chem.AddHs(test)
    
    #test=convert_ase2rdkit(pybel_opt(smile,10))
    
    return AllChem.EmbedMolecule(test, maxAttempts=10, randomSeed=42)
    
def rdkit_opt(smile,steps):
    m = Chem.MolFromSmiles(smile)
    m = Chem.AddHs(m)
    
    AllChem.EmbedMolecule(m, maxAttempts=20, randomSeed=42)
    AllChem.UFFOptimizeMolecule(m,maxIters=steps)
    
    return rdkit2ase(m)

def mol_from_smiles(smile,steps=200):
    if try_rdkit(smile)==0:
        #print("Going for rdkit")
        return rdkit_opt(smile,steps)
    else:
        #print("Going for pybel")
        return pybel_opt(smile,steps)

In [ ]:
# Example smiles

smi="C1(C(C(C2=C(C1=CC=C3C4=CC=C5)C3=C6C7=C2C8=CC9=C7C%10=C%11C(C%12=C%13C%14=C%15C(C(C%16=C%17C(C%18=C%19C%20=C%21C(C(C%22=CC=CC%23=C%22C%21=C(C%19=C%17C%24=C%25)C%25=C%23)=CC=C%26%27)=C%26C%28=C%20C(C(C(C%28=C(C%27=C/%29)NC%29=C/%30)=N/%31)=CC%31=C\C%32=CC=C(/C=C%33C=CC%30=N/%33)N%32)=C%18)=CC%34=C%16C%15=C(C%13=C%11C9=C%35)C%35=C%34)=C%24C=C%36%37)=C%36C%38=C%14C(C(C(C%38=C(C%37=C/%39)NC%39=C/%40)=N/%41)=CC%41=C\C%42=CC=C(/C=C%43C=CC%40=N/%43)N%42)=C%12)=CC%44=C%10C6=C4C5=C%44)=C(C8=C/%45)NC%45=C/%46)=N/%47)=CC%47=C\C%48=CC=C(/C=C%49C=CC%46=N/%49)N%48"
smi2="C12=CC=CC3=C1C(CC4=C3C5=C(C(C(C6=CC=CC7=C6C(C8)=CC=C7)=C8C=C9)=C9C=C%10)C%10=CC=C5C=C4)=CC=C2"
smi1="C=C1C(C=C(C=C2)C3=CC=C(C4=CC(C=C5)=C(C6=C5C=C(C7=CC=C(C8=CC9=C(C%10=C(C9=C)C=CC=C%10)C=C8C%11=C)C%11=C7)C=C6)C=C4C=C%12)C%12=C3)=C2C(C=C%13%14)=C1C=C%13C%15=CC=CC=C%15C%14=C"

In [ ]:
from ase.visualize import view
mol=mol_from_smiles(smi2)
#view(mol)

In [ ]:
view(mol)

## ADD GOLD

The following cell is just to place the molecule on top of a predefined Gold slab. Skip it if you don't need it or improve if you need a better slab

In [ ]:
smimol=mol_from_smiles(smi2)
from ase.io import read
gold_slab=read("examples/Au896.xyz")

# get cm gol
cm_gold=gold_slab.get_center_of_mass()
# flatten the mol
#smimol.positions[:,2]=0
# get mol cm
cm_mol=smimol.get_center_of_mass()
# get the diff
diff=cm_gold-cm_mol
# shit the mol
smimol.positions+=diff
# shift on top
smimol.positions[:,2]+=(max(gold_slab.positions[:,2])-smimol.get_center_of_mass()[2])+3.2

# join the slab and the mol

mol = Atoms()
for atm in gold_slab:
    mol.append(atm)
for atm in smimol:
    mol.append(atm)

In [ ]:
write("mol-rdkit.pdb",mol)

## OPTIMIZE THE MOLECULE USING XTB ( or other semiempirical methods)

In [ ]:
from ase.build import molecule
from xtb.ase.calculator import XTB

In [ ]:
mol.pbc=False
#newstrxtb.calc=XTB(method="GFN2-xTB")
# GFNFF is very fast. See here for other methods:
# https://xtb-python.readthedocs.io/en/latest/general-api.html#available-calculation-methods
mol.calc=XTB(method="GFNFF")

Evaluate the following if you want to fix some atoms, e.g. the metal substrate.

In [ ]:
from ase.constraints import FixAtoms

fix_z=max(mol[np.where(np.asarray(mol.get_chemical_symbols())=="Au")[0]].positions[:,2])
idatoms_tobefixed = np.where(mol.positions[:,2]<fix_z)[0]
atoms_tobefixed = FixAtoms(idatoms_tobefixed)
mol.set_constraint(atoms_tobefixed)

In [ ]:
from ase.optimize.lbfgs import LBFGS

hareV=27.211399

opt = LBFGS(mol)
opt.run(fmax=0.00001)
forces = mol.get_forces()
energy = mol.get_potential_energy()
positions = mol.get_positions()

#
print("Final energy is", energy, "eV")
print("Final energy is", energy/hareV, "hartree")

#print("All done, final positions are:")
#print(positions)

In [ ]:
view(mol)

In [ ]:
write("mol-optimized.pdb",mol)

## Compute energy and forces using DFT and Beyond-DFT

In [ ]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np

In [ ]:
atoms = molecule('H2O')

#atoms = Atoms(mol.get_chemical_symbols(), positions=mol.positions)

# See here for the methods:
# http://www.psicode.org/psi4manual/master/autodoc_dft_energy.html#table-energy-dft

# here the basisets:
# http://www.psicode.org/psi4manual/master/basissets_byfamily.html


# and check also this link for a nice overview of the Psi4 calculator in ASE:
# https://gitlab.com/ase/ase/blob/master/doc/ase/calculators/psi4.rst
calc = Psi4(atoms = atoms,
        method = 'b3lyp', #'b3lyp'
        memory = '16000MB', # the default is 500MB, be aware!
        basis = '6-311g_d_p_',
        num_threads="max"
           )

atoms.calc = calc
print(atoms.get_potential_energy())
print(atoms.get_forces())
